In [1]:
import logging
from conf import LisaLogging
LisaLogging.setup()

# Comment the follwing line to disable devlib debugging statements
logging.getLogger('ssh').setLevel(logging.DEBUG)

In [2]:
import json
import time
import os

# Test environment setup

In [3]:
# Setup a target configuration
conf = {

    # Platform and board to target
    "platform"    : "linux",
    "board"       : "juno",

    # Login credentials
    "host"        : "192.168.0.1",
    "username"    : "root",
    "password"    : "",

    # Local installation path
    "tftp"  : {
        "folder"    : "/var/lib/tftpboot",
        "kernel"    : "kern.bin",
        "dtb"       : "dtb.bin",
    },

    # Tools to deploy
    "tools" : [ "rt-app", "taskset" ],

    # RTApp calibration values (comment to let LISA do a calibration run)
    "rtapp-calib" :  {
        "0": 358, "1": 138, "2": 138, "3": 357, "4": 359, "5": 355
    },

    # FTrace configuration
    "ftrace" : {
         "events" : [
             "cpu_idle",
             "sched_switch",
         ],
         "buffsize" : 10240,
    },
    
    # Where results are collected
    "results_dir" : "TestEnvExample",
    
    # Tune which devlib module are required
    #"exclude_modules" : [ "hwmon" ],

}

In [4]:
from env import TestEnv

# Initialize a test environment using the provided configuration
te = TestEnv(conf)

05:59:27  INFO    :         Target - Using base path: /home/derkling/Code/lisa
05:59:27  INFO    :         Target - Loading custom (inline) target configuration
05:59:27  INFO    :         Target - Devlib modules to load: ['bl', 'hwmon', 'cpufreq']
05:59:27  INFO    :         Target - Connecting linux target with: {'username': 'root', 'host': '192.168.0.1', 'password': ''}
05:59:27  DEBUG   : Logging in root@192.168.0.1
06:00:19  DEBUG   : id
06:00:19  DEBUG   : echo $PATH
06:00:20  DEBUG   : ls -1 /usr/local/bin
06:00:20  DEBUG   : cat /proc/cpuinfo
06:00:20  DEBUG   : sudo -- sh -c 'dmidecode -s system-version'
06:00:21  DEBUG   : if [ -e '/sys/class/hwmon' ]; then echo 1; else echo 0; fi
06:00:21  DEBUG   : ls -1 /sys/class/hwmon
06:00:22  DEBUG   : if [ -e '/sys/class/hwmon/hwmon0/name' ]; then echo 1; else echo 0; fi
06:00:22  DEBUG   : sudo -- sh -c 'cat '\''/sys/class/hwmon/hwmon0/name'\'''
06:00:23  DEBUG   : ls -1 /sys/class/hwmon/hwmon0/
06:00:23  DEBUG   : sudo -- sh -c 'cat

## Attributes

The initialization of the test environment pre-initialize some useful<br>
environment variables which are available to write test cases.

These are some of the information available via the TestEnv object.

In [5]:
# The complete configuration of the target we have configured
print json.dumps(te.conf, indent=4)

{
    "username": "root", 
    "ftrace": {
        "buffsize": 10240, 
        "events": [
            "cpu_idle", 
            "sched_switch"
        ]
    }, 
    "rtapp-calib": {
        "1": 138, 
        "0": 358, 
        "3": 357, 
        "2": 138, 
        "5": 355, 
        "4": 359
    }, 
    "host": "192.168.0.1", 
    "password": "", 
    "tools": [
        "rt-app", 
        "taskset", 
        "trace-cmd", 
        "taskset", 
        "trace-cmd", 
        "perf", 
        "cgroup_run_into.sh"
    ], 
    "results_dir": "TestEnvExample", 
    "platform": "linux", 
    "board": "juno", 
    "__features__": [], 
    "tftp": {
        "kernel": "kern.bin", 
        "folder": "/var/lib/tftpboot", 
        "dtb": "dtb.bin"
    }
}


In [6]:
# Last configured kernel and DTB image
print te.kernel
print te.dtb

None
None


In [7]:
# The IP and MAC address of the target
print te.ip
print te.mac

192.168.0.1
None


In [8]:
# A full platform descriptor
print json.dumps(te.platform, indent=4)

{
    "nrg_model": {
        "big": {
            "cluster": {
                "nrg_max": 64
            }, 
            "cpu": {
                "cap_max": 1024, 
                "nrg_max": 616
            }
        }, 
        "little": {
            "cluster": {
                "nrg_max": 57
            }, 
            "cpu": {
                "cap_max": 447, 
                "nrg_max": 93
            }
        }
    }, 
    "clusters": {
        "big": [
            1, 
            2
        ], 
        "little": [
            0, 
            3, 
            4, 
            5
        ]
    }, 
    "cpus_count": 6, 
    "freqs": {
        "big": [
            450000, 
            625000, 
            800000, 
            950000, 
            1100000
        ], 
        "little": [
            450000, 
            575000, 
            700000, 
            775000, 
            850000
        ]
    }, 
    "topology": [
        [
            0, 
            3, 
            4, 
        

In [9]:
# A pre-created folder to host the tests results generated using this
# test environment, notice that the suite could add additional information
# in this folder, like for example a copy of the target configuration
# and other target specific collected information
te.res_dir

'/home/derkling/Code/lisa/results/TestEnvExample'

In [10]:
# The working directory on the target
te.workdir

'/data/local/schedtest'

In [11]:
# The target topology, which can be used to build BART assertions
te.topology

cluster [[0, 3, 4, 5], [1, 2]]
cpu [[0], [1], [2], [3], [4], [5]]

## Functions

Some methods are also exposed to test developers which could be used to easy
the creation of tests.

These are some of the methods available:

In [12]:
# Calibrate RT-App (if required) and get the most updated calibration value
te.calibration()

{0: 358, 1: 138, 2: 138, 3: 357, 4: 359, 5: 355}

In [13]:
# Generate a JSON file with the complete platform description
te.platform_dump(dest_dir='/tmp')

({'clusters': {'big': [1, 2], 'little': [0, 3, 4, 5]},
  'cpus_count': 6,
  'freqs': {'big': [450000, 625000, 800000, 950000, 1100000],
   'little': [450000, 575000, 700000, 775000, 850000]},
  'nrg_model': {u'big': {u'cluster': {u'nrg_max': 64},
    u'cpu': {u'cap_max': 1024, u'nrg_max': 616}},
   u'little': {u'cluster': {u'nrg_max': 57},
    u'cpu': {u'cap_max': 447, u'nrg_max': 93}}},
  'topology': [[0, 3, 4, 5], [1, 2]]},
 '/tmp/platform.json')

In [14]:
# Force a reboot of the target (and wait specified [s] before reconnect)
te.reboot(reboot_time=60, ping_time=15)

06:01:08  INFO    :   HostResolver - Target (00:02:f7:00:5a:5b) at IP address: 192.168.0.1
06:01:08  DEBUG   : sudo -- sh -c 'sleep 2 && reboot -f &'
06:01:08  INFO    :         Reboot - Waiting up to 60[s] for target [192.168.0.1] to reboot...
06:02:13  INFO    :         Target - Devlib modules to load: ['bl', 'hwmon', 'cpufreq']
06:02:13  INFO    :         Target - Connecting linux target with: {'username': 'root', 'host': '192.168.0.1', 'password': ''}
06:02:13  DEBUG   : Logging in root@192.168.0.1
06:02:14  DEBUG   : id
06:02:15  DEBUG   : echo $PATH
06:02:15  DEBUG   : ls -1 /usr/local/bin
06:02:16  DEBUG   : cat /proc/cpuinfo
06:02:16  DEBUG   : sudo -- sh -c 'dmidecode -s system-version'
06:02:16  DEBUG   : if [ -e '/sys/class/hwmon' ]; then echo 1; else echo 0; fi
06:02:17  DEBUG   : ls -1 /sys/class/hwmon
06:02:17  DEBUG   : if [ -e '/sys/class/hwmon/hwmon0/name' ]; then echo 1; else echo 0; fi
06:02:18  DEBUG   : sudo -- sh -c 'cat '\''/sys/class/hwmon/hwmon0/name'\'''
06:02

In [15]:
# Resolve a MAC address into an IP address
te.resolv_host(host='00:02:F7:00:5A:5B')

06:03:00  INFO    :   HostResolver - Target (00:02:F7:00:5A:5B) at IP address: 192.168.0.1


('00:02:F7:00:5A:5B', '192.168.0.1')

In [16]:
# Copy the specified file into the TFTP server folder defined by configuration
te.tftp_deploy('/etc/group')

06:03:00  INFO    :           TFTP - Deploy /etc/group into /var/lib/tftpboot/group


A special TestEnv attribute is <b>target</b>, which represent a <b>devlib instance</b>.<br>
Using the target attribute we can access to the full set of devlib provided<br>
functionalities. Which are summarized in the following sections.


# Access to the devlib API

In [17]:
# Run a command on the target
te.target.execute("echo -n 'Hello Test Environment'", as_root=False)

06:03:01  DEBUG   : echo -n 'Hello Test Environment'


'Hello Test Environment'

In [18]:
# Spawn a command in background on the target
te.target.kick_off("sleep 10", as_root=True)

06:03:02  DEBUG   : sudo -- sh -c 'sh -c "sleep 10" 1>/dev/null 2>/dev/null &'


''

In [19]:
# Acces to many target specific information
print "ABI                 : ", te.target.abi
print "big Core Family     : ", te.target.big_core
print "LITTLE Core Family  : ", te.target.little_core
print "CPU's Clusters IDs  : ", te.target.core_clusters
print "CPUs type           : ", te.target.core_names

ABI                 :  arm64
big Core Family     :  A57
LITTLE Core Family  :  A53
CPU's Clusters IDs  :  [0, 1, 1, 0, 0, 0]
CPUs type           :  ['A53', 'A57', 'A57', 'A53', 'A53', 'A53']


In [20]:
# Access to big.LITTLE specific information
print "big CPUs IDs        : ", te.target.bl.bigs
print "LITTLE CPUs IDs     : ", te.target.bl.littles
print "big CPUs freqs      : {}".format(te.target.bl.get_bigs_frequency())
print "big CPUs governor   : {}".format(te.target.bl.get_bigs_governor())

06:03:02  DEBUG   : sudo -- sh -c 'cat '\''/sys/devices/system/cpu/online'\'''
06:03:03  DEBUG   : sudo -- sh -c 'cat '\''/sys/devices/system/cpu/cpu1/cpufreq/scaling_cur_freq'\'''


big CPUs IDs        :  [1, 2]
LITTLE CPUs IDs     :  [0, 3, 4, 5]
big CPUs freqs      : 1100000

06:03:03  DEBUG   : sudo -- sh -c 'cat '\''/sys/devices/system/cpu/online'\'''
06:03:04  DEBUG   : sudo -- sh -c 'cat '\''/sys/devices/system/cpu/cpu1/cpufreq/scaling_governor'\'''



big CPUs governor   : performance


# Sample energy from the target

In [21]:
# Reset and sample energy counters
te.emeter.reset()
nrg = te.emeter.sample()
nrg = json.dumps(te.emeter.sample(), indent=4)
print "First read: ", nrg
time.sleep(2)
nrg = te.emeter.sample()
nrg = json.dumps(te.emeter.sample(), indent=4)
print "Second read: ", nrg

06:03:04  DEBUG   : sudo -- sh -c 'cat '\''/sys/class/hwmon/hwmon13/energy1_input'\'''
06:03:05  DEBUG   : sudo -- sh -c 'cat '\''/sys/class/hwmon/hwmon14/energy1_input'\'''
06:03:05  DEBUG   : sudo -- sh -c 'cat '\''/sys/class/hwmon/hwmon13/energy1_input'\'''
06:03:05  DEBUG   : sudo -- sh -c 'cat '\''/sys/class/hwmon/hwmon14/energy1_input'\'''
06:03:06  DEBUG   : sudo -- sh -c 'cat '\''/sys/class/hwmon/hwmon13/energy1_input'\'''
06:03:06  DEBUG   : sudo -- sh -c 'cat '\''/sys/class/hwmon/hwmon14/energy1_input'\'''
06:03:09  DEBUG   : sudo -- sh -c 'cat '\''/sys/class/hwmon/hwmon13/energy1_input'\'''
06:03:09  DEBUG   : sudo -- sh -c 'cat '\''/sys/class/hwmon/hwmon14/energy1_input'\'''
06:03:10  DEBUG   : sudo -- sh -c 'cat '\''/sys/class/hwmon/hwmon13/energy1_input'\'''
06:03:10  DEBUG   : sudo -- sh -c 'cat '\''/sys/class/hwmon/hwmon14/energy1_input'\'''


First read:  {
    "a53": {
        "total": 0.37534899999999993, 
        "last": 51.372362, 
        "delta": 0.18520200000000386
    }, 
    "a57": {
        "total": 1.0124409999999955, 
        "last": 59.688784, 
        "delta": 0.5027259999999956
    }
}
Second read:  {
    "a53": {
        "total": 1.7191619999999972, 
        "last": 52.716175, 
        "delta": 0.17561400000000305
    }, 
    "a57": {
        "total": 2.4685759999999988, 
        "last": 61.144919, 
        "delta": 0.4992990000000006
    }
}


# Configure FTrace for a sepcific experiment

In [22]:
# Configure a specific set of events to trace
te.ftrace_conf(
    {                                                                                                                                             
         "events" : [                                                                                                                                            
             "cpu_idle",                                                                                                                                         
             "cpu_capacity",
             "cpu_frequency",
             "sched_switch",
         ],                                                                                                                                                      
         "buffsize" : 10240                                                                                                                                      
    }
)

06:03:10  DEBUG   : /usr/bin/scp -r   /home/derkling/Code/lisa/libs/devlib/devlib/bin/arm64/trace-cmd root@192.168.0.1:/root/devlib-target/bin/trace-cmd
06:03:11  DEBUG   : chmod a+x /root/devlib-target/bin/trace-cmd
06:03:11  DEBUG   : cat /sys/kernel/debug/tracing/available_events
06:03:11  INFO    :         FTrace - Enabled events:
06:03:11  INFO    :         FTrace -   ['cpu_idle', 'cpu_capacity', 'cpu_frequency', 'sched_switch']
06:03:11  INFO    :         FTrace -   None


In [23]:
# Start/Stop a FTrace session
te.ftrace.start()
te.target.execute("uname -a")
te.ftrace.stop()

06:03:11  DEBUG   : sudo -- sh -c 'echo 10240 > '\''/sys/kernel/debug/tracing/buffer_size_kb'\'''
06:03:12  DEBUG   : sudo -- sh -c 'cat '\''/sys/kernel/debug/tracing/buffer_size_kb'\'''
06:03:12  DEBUG   : sudo -- sh -c '/root/devlib-target/bin/trace-cmd reset'
06:03:13  DEBUG   : sudo -- sh -c '/root/devlib-target/bin/trace-cmd start -e cpu_idle -e cpu_capacity -e cpu_frequency -e sched_switch'
06:03:14  DEBUG   : sudo -- sh -c 'echo TRACE_MARKER_START > '\''/sys/kernel/debug/tracing/trace_marker'\'''
06:03:15  DEBUG   : sudo -- sh -c '/root/devlib-target/bin/shutils cpufreq_trace_all_frequencies'
06:03:15  DEBUG   : uname -a
06:03:15  DEBUG   : sudo -- sh -c '/root/devlib-target/bin/shutils cpufreq_trace_all_frequencies'
06:03:16  DEBUG   : sudo -- sh -c 'echo TRACE_MARKER_STOP > '\''/sys/kernel/debug/tracing/trace_marker'\'''
06:03:16  DEBUG   : sudo -- sh -c '/root/devlib-target/bin/trace-cmd stop'


In [24]:
# Collect and visualize the trace
trace_file = os.path.join(te.res_dir, 'trace.dat')
te.ftrace.get_trace(trace_file)
output = os.popen("DISPLAY=:0.0 kernelshark {}".format(trace_file))

06:03:17  DEBUG   : sudo -- sh -c '/root/devlib-target/bin/trace-cmd extract -o /root/devlib-target/trace.dat'
06:03:18  DEBUG   : /usr/bin/scp -r   root@192.168.0.1:/root/devlib-target/trace.dat /home/derkling/Code/lisa/results/TestEnvExample/trace.dat
